> You can open this notebook in Colab by clicking the Colab icon. Colab provides GPU for free. You can also run this notebook locally by installing the dependencies listed in `requirements.txt`.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/osbm/unet_explainer/blob/main/tutorial.ipynb)

### Preamble

The second part of this hands-on will allow you to study in depth the structure and performance of the popular deep learning U-Net architecture, in the context of 2D multi-slice prostate MR images segmentation. You will use the (prostate-158 train dataset)[https://zenodo.org/record/6481141] (139 mri images) and (prostate-158 test dataset)[https://zenodo.org/record/6592345] dataset (19 mri images).

### Objectives

* Study how to code a deep learning segmentation method with the pytorch library.
* Compare the performance of a U-Net model based on the choice of key hyper-parameters.


## Reminder on the U-Net architecture

U-Net is based on a two-stage convolutional network architecture. The first part, known as the encoder, is similar to conventional CNNs and extracts high-level information. The second part is the decoder, which uses information from the encoder and applies a set of convolutions and upsampling operations to gradually transform feature maps with the purpose of reconstructing segmentation maps at the resolution of the imput image. U-Net architecture also integrates skip connections between the encoder and decoder parts with the goal of retrieving details that were potentially lost during the downsampling while also stabilizing the learning procedure. An illustration of the network architecture is given below.


![unet-architecture](./assets/unet-architecture.png)

The U-Net architecture can be defined through the following main parameters:
- the number of feature maps at the first level
- the number of levels
- the use of the batch normalizations at each level
- the type of activation functions
- the use of dropout operations
- the use of data augmentation

The performance of deep learning model also depends on the optimization conditions that were used during the learning process, the main ones being:
- the optimization algorithm (*ADAM* and *RMSprop* being among the most popular)
- the learning rate


In [ ]:
!pip install git+https://github.com/osbm/unet_explainer.git


In [ ]:
from unet_pytorch import ProstateDataset, get_transforms, get_parameter_number, fit_model, predict, set_seed
from torch.utils.data import DataLoader
import torch
from torch import nn
from monai.networks.nets import UNet
import monai

In [ ]:
set_seed(42)

In [ ]:
train_transforms, valid_transforms = get_transforms(image_size=256)

train_ds = ProstateDataset(folder='train', transform=train_transforms)
valid_ds = ProstateDataset(folder='valid', transform=valid_transforms)
test_ds = ProstateDataset(folder='test', transform=valid_transforms)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_ds, batch_size=16, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=16, shuffle=False)


In [ ]:
train_transforms

In [ ]:
model = UNet(
    spatial_dims=2, # 3d image
    in_channels=1,  # we only used  T2 weighed MRI images
    out_channels=3, # 3 labels
    channels=[16, 32, 64, 128, 256, 512],
    strides=(2, 2, 2, 2, 2), # CNN strides
    num_res_units=4, # residual connections
    dropout=0.15, # dropout rate
)
get_parameter_number(model)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), 1e-3)
loss = monai.losses.DiceLoss(to_onehot_y=True, softmax=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)


model, history = fit_model(
    model=model,
    train_loader=train_loader,
    valid_loader=valid_loader,
    optimizer=optimizer,
    loss=loss,
    device=device,
    epochs=10,
)


In [ ]:
from unet_pytorch import plot_history

plot_history(history)
